In [10]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

record_path = "/home/wc-gpu/MasterThesis/models/research/object_detection/data/generic_prod_session_train.record"

In [35]:
from object_detection.utils import visualization_utils as vu
from object_detection.protos import string_int_label_map_pb2 as pb
from object_detection.data_decoders.tf_example_decoder import TfExampleDecoder as TfDecoder
from google.protobuf import text_format

def main(tfrecords_filename, label_map=None):
    def _parse_record(tf_record):
            features = {
                'image': tf.FixedLenFeature([], dtype=tf.string),
                'label': tf.FixedLenFeature([], dtype=tf.int64)
            }
            record = tf.parse_single_example(tf_record, features)

            image_raw = tf.decode_raw(record['image'], tf.float32)
            image_raw = tf.reshape(image_raw, shape=(224, 224, 3))

            label = tf.cast(record['label'], tf.int32)
            label = tf.one_hot(label, depth=2)

            return { 'image': image_raw }, label
    class_dict = {}
    for entry in label_map.values():
        class_dict[entry["id"]] = {'name':entry["name"]}
    sess = tf.Session()
    sess.run(tf.tables_initializer())
    for record in tf.python_io.tf_record_iterator(tfrecords_filename):
        record = _parse_record(record)
        scores = np.ones(record['groundtruth_boxes'].shape[0])
        vu.visualize_boxes_and_labels_on_image_array( 
            record['image'],                                               
            record['groundtruth_boxes'],                                                     
            record['groundtruth_classes'],
            scores,
            class_dict,
            max_boxes_to_draw=None,
            use_normalized_coordinates=True)

In [36]:
main(record_path, label_map = {1: {'id': 1, 'name': 'obj'}})

TypeError: tuple indices must be integers or slices, not str